In [ ]:

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm  # 진행률 표시를 위한 라이브러리

# qwe.csv 파일 읽기
df = pd.read_csv('qwe.csv')

def check_business_number(business_number):
    driver = webdriver.Chrome()
    try:
        # 홈택스 사이트 접속
        driver.get('https://hometax.go.kr/websquare/websquare.html?w2xPath=/ui/pp/index_pp.xml&tmIdx=43&tm2lIdx=4306000000&tm3lIdx=4306080000')
        
        # 페이지 로딩 대기
        time.sleep(1.75)
        
        # 사업자등록번호 입력
        input_field = driver.find_element(By.ID, 'mf_txppWframe_bsno')
        input_field.clear()
        input_field.send_keys(business_number)
        
        # 조회 버튼 클릭
        search_button = driver.find_element(By.ID, 'mf_txppWframe_trigger5')
        search_button.click()
        
        # 결과 대기 및 가져오기
        wait = WebDriverWait(driver, 10)
        result = wait.until(EC.presence_of_element_located((By.ID, 'mf_txppWframe_grid2_cell_0_1')))
        return result.text
        
    except Exception as e:
        print(f"Error checking business number {business_number}: {str(e)}")
        return "조회 실패"
    
    finally:
        driver.quit()

def check_business_numbers(df):
    results = []
    driver = webdriver.Chrome()
    
    try:
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            business_number = str(row['사업장등록번호'])
            store_name = row['현장명']
            
            try:
                # 홈택스 사이트 접속
                driver.get('https://hometax.go.kr/websquare/websquare.html?w2xPath=/ui/pp/index_pp.xml&tmIdx=43&tm2lIdx=4306000000&tm3lIdx=4306080000')
                time.sleep(1.75)  # 페이지 로딩 대기
                
                # 사업자등록번호 입력
                input_field = driver.find_element(By.ID, 'mf_txppWframe_bsno')
                input_field.clear()
                input_field.send_keys(business_number)
                
                # 조회 버튼 클릭
                search_button = driver.find_element(By.ID, 'mf_txppWframe_trigger5')
                search_button.click()
                
                # 결과 대기 및 가져오기
                wait = WebDriverWait(driver, 10)
                result = wait.until(EC.presence_of_element_located((By.ID, 'mf_txppWframe_grid2_cell_0_1')))
                status = result.text
                
                results.append({
                    '현장명': store_name,
                    '사업장등록번호': business_number,
                    '조회결과': status
                })
                
                # 서버 부하 방지를 위한 대기
                time.sleep(1.75)
                
            except Exception as e:
                print(f"\n{store_name}({business_number}) 조회 중 오류: {str(e)}")
                results.append({
                    '현장명': store_name,
                    '사업장등록번호': business_number,
                    '조회결과': '조회 실패'
                })
                time.sleep(1.75)
                
    finally:
        driver.quit()
        
    return pd.DataFrame(results)

# 전체 데이터 조회 실행
print("전체 사업자등록번호 조회를 시작합니다...")
results_df = check_business_numbers(df)

# 결과 저장
timestamp = time.strftime("%Y%m%d_%H%M%S")
results_df.to_csv(f'business_status_{timestamp}.csv', index=False, encoding='utf-8-sig')
print(f"조회가 완료되었습니다. 결과가 business_status_{timestamp}.csv 파일에 저장되었습니다.")


In [2]:
import pandas as pd
import glob
import os

# 가장 최근의 business_status 파일 찾기
files = glob.glob('business_status_*.csv')
latest_file = max(files, key=os.path.getctime)

# CSV 파일 읽기
df = pd.read_csv(latest_file)

# '휴업' 또는 '폐업'이 포함된 행 필터링
closed_businesses = df[df['조회결과'].str.contains('휴업|폐업', na=False)]

# 필요한 컬럼 선택 (사업장등록번호 추가)
result = closed_businesses[['현장명', '사업장등록번호', '조회결과']]

# 결과 저장
timestamp = time.strftime("%Y%m%d_%H%M%S")
output_filename = f'closed_businesses_{timestamp}.csv'
result.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"총 {len(result)}개의 휴/폐업 업체가 발견되었습니다.")
print(f"결과가 {output_filename}에 저장되었습니다.")
display(result)  # 주피터 노트북에서 결과 표시

In [5]:
import os
import glob

# 삭제할 CSV 파일 패턴들
patterns = [
    'business_status_*.csv',
    'closed_businesses_*.csv'
]

deleted_files = []
for pattern in patterns:
    files = glob.glob(pattern)
    for file in files:
        try:
            os.remove(file)
            deleted_files.append(file)
        except Exception as e:
            print(f"파일 {file} 삭제 실패: {str(e)}")

if deleted_files:
    print("삭제된 파일 목록:")
    for file in deleted_files:
        print(f"- {file}")
else:
    print("삭제할 파일이 없습니다.")